# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [72]:
#Read CSV
file = os.path.join("..","output_data","Cities.csv")
cities_df = pd.read_csv(file)

cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dauphin,51.15,-100.05,50.00,71,1,5.82,CA,1600045433
1,rivera,-30.91,-55.55,61.90,95,92,8.86,UY,1600045317
2,talnakh,69.49,88.40,50.00,100,40,11.18,RU,1600045434
3,rikitea,-23.12,-134.97,74.05,84,13,17.27,PF,1600045254
4,bambous virieux,-20.26,57.41,69.80,77,75,10.29,MU,1600045434


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [73]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Using lat and lng as locations
locations = cities_df[["Lat","Lng"]]

#Using humidity as weight
weight = cities_df["Humidity"]
maxhum = cities_df["Humidity"].max()


fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations,weights=weight, max_intensity = maxhum)
fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

In [74]:
cities_df.count()

City          558
Lat           558
Lng           558
Max Temp      558
Humidity      558
Cloudiness    558
Wind Speed    558
Country       554
Date          558
dtype: int64

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
ideal_cities = cities_df.loc[(cities_df["Max Temp"] >= 50) & (cities_df["Max Temp"] <= 85) & (cities_df["Humidity"] < 40) & (cities_df["Cloudiness"] < 10)]
ideal_cities.dropna(inplace = True)
ideal_cities.count()

<ipython-input-75-60b3d0eebd1f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities.dropna(inplace = True)


City          23
Lat           23
Lng           23
Max Temp      23
Humidity      23
Cloudiness    23
Wind Speed    23
Country       23
Date          23
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [76]:
hotel_list = []

maps_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_cities.iterrows():
    
    params = {"location": f"{row['Lat']}, {row['Lng']}", "radius": 5000, "type": "lodging", "key": g_key}

    response = requests.get(maps_url, params=params).json()
    try:
        hotel_list.append(response["results"][0]["name"])
    except:
        hotel_list.append("")
ideal_cities["Hotel Name"] = hotel_list
ideal_cities.head()
    

<ipython-input-76-9c328f0b7c25>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities["Hotel Name"] = hotel_list


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,arraial do cabo,-22.84,-46.65,84.20,23,0,3.36,BR,1600045437,Chácara Cantinho do Céu
60,jensen beach,38.72,-112.10,82.00,15,0,5.99,US,1600045446,Hampton Inn Richfield
89,comodoro rivadavia,-13.66,-59.79,77.88,29,0,4.54,BR,1600045434,Romax Hotel
100,jalu,29.03,21.55,81.70,35,0,11.03,LY,1600045455,Jalu Hotel
123,airai,-8.93,125.41,72.10,38,3,6.85,TL,1600045462,Juvinal Martin


In [77]:
hotel_df = ideal_cities.loc[ideal_cities["Hotel Name"] != ""]
hotel_df.head(21)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,arraial do cabo,-22.84,-46.65,84.20,23,0,3.36,BR,1600045437,Chácara Cantinho do Céu
60,jensen beach,38.72,-112.10,82.00,15,0,5.99,US,1600045446,Hampton Inn Richfield
89,comodoro rivadavia,-13.66,-59.79,77.88,29,0,4.54,BR,1600045434,Romax Hotel
100,jalu,29.03,21.55,81.70,35,0,11.03,LY,1600045455,Jalu Hotel
123,airai,-8.93,125.41,72.10,38,3,6.85,TL,1600045462,Juvinal Martin
170,buraydah,26.33,43.98,80.60,28,0,3.36,SA,1600045473,Mövenpick Hotel Qassim
188,tahta,26.77,31.50,80.60,32,0,9.57,EG,1600045477,مجموعة دفاع للمحاماه وتأسيس الشركات
206,sitka,57.05,-135.33,64.40,34,1,6.22,US,1600045481,Westmark Sitka
255,neyshabur,36.21,58.80,55.90,39,0,1.28,IR,1600045495,Amiran Hotel
278,gweta,-20.18,25.23,63.86,28,0,6.17,BW,1600045501,Gweta Lodge


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))